In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/Users/yamauchito_satoshi/Documents/data/kaggle/titanic/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

data_dirname = r'/Users/yamauchito_satoshi/Documents/data/kaggle/titanic'

/Users/yamauchito_satoshi/Documents/data/kaggle/titanic/input/.DS_Store
/Users/yamauchito_satoshi/Documents/data/kaggle/titanic/input/test.csv
/Users/yamauchito_satoshi/Documents/data/kaggle/titanic/input/train.csv
/Users/yamauchito_satoshi/Documents/data/kaggle/titanic/input/gender_submission.csv


In [38]:
train = pd.read_csv(os.path.join(data_dirname, "input/train.csv"))
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
test = pd.read_csv(os.path.join(data_dirname, "input/test.csv"))
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# 前処理

In [9]:
gender_submission = pd.read_csv(os.path.join(data_dirname, "input/gender_submission.csv"))
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [10]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [11]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [12]:
train["Age"] = train["Age"].fillna(train["Age"].mode()[0])
train["Cabin"] = train["Cabin"].fillna(train["Cabin"].mode()[0])
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])

test["Age"] = test["Age"].fillna(test["Age"].mode()[0])
test["Fare"] = test["Fare"].fillna(test["Fare"].mode()[0])
test["Cabin"] = test["Cabin"].fillna(test["Cabin"].mode()[0])

In [13]:
train = pd.get_dummies(train, columns=['Sex', 'Embarked'])
test = pd.get_dummies(test, columns=['Sex', 'Embarked'])

# 予測

In [14]:
#k-最近傍法での予測

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=1)

X_train = train[["Pclass", "Age", "SibSp", "Parch", "Fare", "Sex_female", "Sex_male", "Embarked_C", "Embarked_Q", "Embarked_S"]]
y_train = train["Survived"]

X_test = test[["Pclass", "Age", "SibSp", "Parch", "Fare", "Sex_female", "Sex_male", "Embarked_C", "Embarked_Q", "Embarked_S"]]

clf.fit(X_train, y_train)

# t_test に予測結果を格納
y_test = clf.predict(X_test)

In [39]:
# GBDTでの予測
import xgdboost as xgd

ModuleNotFoundError: No module named 'xgdboost'

In [15]:
# PassengerId を取得
PassengerId = np.array(test["PassengerId"]).astype(int)

# y_test と PassengerId を結合
answer = pd.DataFrame(y_test, PassengerId, columns = ["Survived"])

# # titanic_answer_01.csv としてエクスポート
# answer.to_csv("titanic_answer_01.csv", index_label = ["PassengerId"])

In [36]:
gender_submission["PassengerId"][0]

892

In [35]:
answer

,Survived
892,0
893,0
894,1
895,1
896,0
...,...
1305,0
1306,1
1307,0
1308,0


In [37]:
score =0
for i in np.arange(len(PassengerId)):
    if gender_submission["Survived"][i] == answer["Survived"][gender_submission["PassengerId"][i]]:
        score += 1
score = score/len(PassengerId)
print(score)

0.6100478468899522
